# Experiment 10
- **Model:** `yolov8n` *(Nano)*
- **Dataset:** 5m | 60º
- **Sizes:** large
- **Experiments:**
    1. large


## Init

In [1]:
import os
import shutil
import fnmatch

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.7/914.7 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

## Helper Functions

In [3]:
# Change for different file formats
reference = {
  "small": {
    "suffix": ".S",
    "file": "209"
  },
  "mid": {
    "suffix": ".M",
    "file": "503"
  },
  "large": {
    "suffix": ".L",
    "file": "000"
  }
}

In [4]:
# Clone config files
def copy_config(src_folder, dest_folder):
    """
    Copies files from src_folder to dest_folder, excluding subfolders.

    Args:
        src_folder: The path to the source folder.
        dest_folder: The path to the destination folder.
    """

    try:
        # Ensure destination folder exists
        os.makedirs(dest_folder, exist_ok=True)

        for filename in os.listdir(src_folder):
            src_path = os.path.join(src_folder, filename)
            dest_path = os.path.join(dest_folder, filename)

            if os.path.isfile(src_path):
                shutil.copy2(src_path, dest_path) #copy metadata as well.
                #Use shutil.copy for not copying metadata.
                print(f"Copied: {filename}")
            #else: #optional
                #print(f"Skipped (not a file): {filename}") #optional. Uncomment if you want to see the skipped folders.

        print("✅ Copying complete.")

    except Exception as e:
        print(f"❌ An error occurred: {e}")


In [5]:
# Copy filtered dataset images/labels
def copy_and_filter_folder(src_folder, dest_folder, pattern):
    """
    Copies a folder and files that match the given pattern.
    Alerts the user when a folder or file already exists but *does not* overwrite.
    Creates only what is needed.

    :param src_folder: Path to the source folder.
    :param dest_folder: Path to the destination folder.
    :param pattern: Filename pattern to keep (e.g., "*.txt").
    """
    try:
        # Ensure destination folder exists
        if not os.path.exists(dest_folder):
            print(f"✓ Creating destination folder '{dest_folder}'.\n")
            os.makedirs(dest_folder)
        else:
            print(f"✓ Destination folder '{dest_folder}' already exists.\n")

        # Walk through the source folder
        for root, _, files in os.walk(src_folder):
            relative_path = os.path.relpath(root, src_folder)
            new_root = os.path.join(dest_folder, relative_path)

            if not os.path.exists(new_root):
                print(f"Creating subdirectory '{new_root}'")
                os.makedirs(new_root)
            else:
                print(f"❕Subdirectory '{new_root}' already exists.")
                print("Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.")

            for file in files:
                if fnmatch.fnmatch(file, pattern + "*"):
                    src_file = os.path.join(root, file)
                    dest_file = os.path.join(new_root, file)

                    if not os.path.exists(dest_file):
                        shutil.copy2(src_file, dest_file)  # copy metadata as well
                    else:
                        print(f"❗️File '{dest_file}' already exists. Skipping.")

            print(f" ✓ Copying files complete.\n")
        print("✅ Copying dataset complete.")

    except Exception as e:
        print(f"❌ An error occurred: {e}")

In [6]:
def save_on_cloud(source: str, destination: str):
    """
    Saves a folder to a cloud storage location (e.g., Google Drive in Colab).

    Args:
        source (str): The path to the source folder.
        destination (str): The path to the destination folder (in cloud storage).
    """
    # 0. Input Validation (Assertions)
    assert isinstance(source, str), "Source must be a string."
    assert isinstance(destination, str), "Destination must be a string."

    try:
        # 1. Verify Source Folder
        if not os.path.exists(destination):
            os.makedirs(destination)

        # 2. Copy the Folder
        shutil.copytree(source, destination, dirs_exist_ok=True)
        print("✅ Folder copied successfully:\n  ",source,"\n  -->",destination)

    except Exception as e:
        print(f"❌ An error occurred: {e}")

# Datasets builder

## Importing from Drive

In [7]:
!rm -rf /content/sample_data

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Check if the cloud path is ok and the dataset can be found
!ls /content/drive/MyDrive/YOLO

3.5m.v3i.yolov8  3m60.v1i.yolov8  5m60.v1i.yolov8  5m.v1i.yolov8


In [11]:
drive_path = '/content/drive/MyDrive/YOLO'
drive_datasets_paths = os.listdir(drive_path)
drive_datasets = len(drive_datasets_paths)
if (drive_datasets) > 1:
    print("There are %d dataset options:" % drive_datasets)
else:
    print("Theres is only 1 dataset:")
drive_datasets_paths

There are 4 dataset options:


['5m60.v1i.yolov8', '3.5m.v3i.yolov8', '5m.v1i.yolov8', '3m60.v1i.yolov8']

In [12]:
choose_dataset = 4
index = choose_dataset - 1
model_name = os.listdir(drive_path)[index]
print("Choosed model:", model_name)

Choosed model: 3m60.v1i.yolov8


***Readme:***
*   **Option 1:** is desirable if you need to test many subset combinations in the same session (avoid downloading data twice from the cloud).
*   **Option 3:** is best if you're just going to test one subset combination  (avoid downloading unnecessary data from the cloud).



In [34]:
# Option 1 (download full dataset)
source = f"/content/drive/MyDrive/YOLO/{model_name}"
dest = f"/content/YOLO/{model_name}"
#!mkdir /content/YOLO/
!cp -r $source $dest
src_folder = dest

mkdir: cannot create directory ‘/content/YOLO/’: File exists


In [49]:
#!mkdir /content/YOLO

In [ ]:
source = "/content/YOLO/3m60.v1i.yolov8.S/valid"
dest = "/content/YOLO/3m60.v1i.yolov8.M/valid"
!mv $source $dest

In [115]:
copy_directory(source, dest, overwrite=True)

Directory '/content/YOLO/3m60.v1i.yolov8.S/valid' copied to '/content/YOLO/3m60.v1i.yolov8.M/valid'


True

In [114]:
import shutil
import os

source_dir = source
destination_dir = dest

try:
    shutil.copytree(source_dir, destination_dir)
    print(f"Directory '{source_dir}' copied successfully to '{destination_dir}'")
except FileExistsError:
    print(f"Destination directory '{destination_dir}' already exists.")
    # You can handle this in different ways:
    # - Remove the existing directory and try again:
    # shutil.rmtree(destination_dir)  # Be very careful with this!
    # shutil.copytree(source_dir, destination_dir)
    # - Merge the directories (more complex, requires custom logic)
    # - Prompt the user
except Exception as e:
    print(f"Error copying directory: {e}")


# Example with error handling and overwrite:
def copy_directory(source, destination, overwrite=True):
    try:
        if overwrite and os.path.exists(destination):
            shutil.rmtree(destination)  # Remove existing directory
        shutil.copytree(source, destination)
        print(f"Directory '{source}' copied to '{destination}'")
        return True  # Indicate success
    except FileExistsError:
        print(f"Destination '{destination}' exists. Use overwrite=True to replace.")
        return False  # Indicate failure

Destination directory '/content/YOLO/3m60.v1i.yolov8.M/valid' already exists.


In [26]:
# Option 3 (download just what's needed)
# src_folder = f"{drive_path}/{model}"

## ❌ Build dataset 'small'

### Settings

In [69]:
# CHOOSE THE DATASET YOU WANT TO WORK WITH
choosen_data = 'small' # Options: small / mid / large

In [70]:
suffix = reference[choosen_data]['suffix']
dest_folder = f"/content/YOLO/{model_name}{suffix}"
pattern = reference[choosen_data]['file']

In [71]:
print(f'🆗 SETTING PARAMETERS\n - Model: {model_name}\n - rigin: {src_folder}\n - Destination: {dest_folder}\n - Prefix: {pattern}…')

🆗 SETTING PARAMETERS
 - Model: 3m60.v1i.yolov8
 - rigin: /content/YOLO/3m60.v1i.yolov8
 - Destination: /content/YOLO/3m60.v1i.yolov8.S
 - Prefix: 209…


### Builder

In [72]:
copy_config(src_folder, dest_folder)

Copied: README.dataset.txt
Copied: data.yaml
Copied: README.roboflow.txt
✅ Copying complete.


In [73]:
copy_and_filter_folder(src_folder, dest_folder, pattern)

✓ Destination folder '/content/YOLO/3m60.v1i.yolov8.S' already exists.

❕Subdirectory '/content/YOLO/3m60.v1i.yolov8.S/.' already exists.
Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3m60.v1i.yolov8.S/valid'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3m60.v1i.yolov8.S/valid/images'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3m60.v1i.yolov8.S/valid/labels'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3m60.v1i.yolov8.S/train'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3m60.v1i.yolov8.S/train/images'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3m60.v1i.yolov8.S/train/labels'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3m60.v1i.yolov8.S/.ipynb_checkpoints'
 ✓ Copying files complete.

✅ Copying dataset complete.


## ❌ Build dataset 'medium'

### Settings

In [74]:
# CHOOSE THE DATASET YOU WANT TO WORK WITH
choosen_data = 'mid' # Options: small / mid / large

In [75]:
suffix = reference[choosen_data]['suffix']
dest_folder = f"/content/YOLO/{model_name}{suffix}"
pattern = reference[choosen_data]['file']

In [76]:
print(f'🆗 SETTING PARAMETERS\n - Model: {model_name}\n - rigin: {src_folder}\n - Destination: {dest_folder}\n - Prefix: {pattern}…')

🆗 SETTING PARAMETERS
 - Model: 3m60.v1i.yolov8
 - rigin: /content/YOLO/3m60.v1i.yolov8
 - Destination: /content/YOLO/3m60.v1i.yolov8.M
 - Prefix: 503…


### Builder

In [77]:
copy_config(src_folder, dest_folder)

Copied: README.dataset.txt
Copied: data.yaml
Copied: README.roboflow.txt
✅ Copying complete.


In [78]:
copy_and_filter_folder(src_folder, dest_folder, pattern)

✓ Destination folder '/content/YOLO/3m60.v1i.yolov8.M' already exists.

❕Subdirectory '/content/YOLO/3m60.v1i.yolov8.M/.' already exists.
Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3m60.v1i.yolov8.M/valid'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3m60.v1i.yolov8.M/valid/images'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3m60.v1i.yolov8.M/valid/labels'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3m60.v1i.yolov8.M/train'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3m60.v1i.yolov8.M/train/images'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3m60.v1i.yolov8.M/train/labels'
 ✓ Copying files complete.

Creating subdirectory '/content/YOLO/3m60.v1i.yolov8.M/.ipynb_checkpoints'
 ✓ Copying files complete.

✅ Copying dataset complete.


## Remove datasets (restart)

In [44]:
# Removes complete local dataset
#!rm -rf /content/YOLO

# Model constructor
Model: `yolov8n` *(Nano)*



## Download model

In [9]:
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolov8n.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 285MB/s]


# Finetuning

### Info

In [61]:
!nvidia-smi

Wed Feb 12 17:55:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [62]:
!yolo version

8.3.74


-----
## Experiment 10 *(full dataset)*
### *5m | 60º | large*

### Train

In [ ]:

# Train the model
model.train(
    data=f"/content/YOLO/{model_name}/data.yaml",
    epochs=100,
    imgsz=240,
    freeze=10,
    batch=64
)


### Validation

In [ ]:
# Load the trained YOLO model
model1 = YOLO("/content/runs/detect/train/weights/best.pt")

# Validate the model
model1.val(data=f"/content/YOLO/{model_name}/data.yaml")

### Inference

#### Test 1 (small weed)

In [84]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [ ]:
# Ejecutar predicción sobre la imagen
results = model1(image, save=True, project="model10(5_60_L)", name="experiment1")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")

In [86]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.875 0.85 0.25416666666666665 0.125
0 0.9041666666666667 0.8875 0.19583333333333333 0.13333333333333333
0 0.9625 0.9458333333333333 0.07916666666666666 0.08333333333333333
0 0.7083333333333334 0.8791666666666667 0.30833333333333335 0.06666666666666667
0 0.4375 0.8458333333333333 0.20833333333333334 0.041666666666666664
0 0.4041666666666667 0.825 0.225 0.041666666666666664
0 0.3333333333333333 0.7583333333333333 0.21666666666666667 0.09166666666666666
0 0.2375 0.7541666666666667 0.225 0.041666666666666664
0 0.09583333333333334 0.7291666666666666 0.19166666666666668 0.09166666666666666



#### Test 2 (medium weed)

In [89]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [ ]:
# Ejecutar predicción sobre la imagen
results = model1(image, save=True, project="model10(5_60_L)", name="experiment2")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")

In [91]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.05416666666666667 0.7958333333333333 0.10416666666666667 0.13333333333333333
0 0.0875 0.6625 0.175 0.275
0 0.25 0.575 0.5 0.125
0 0.09166666666666666 0.5583333333333333 0.10833333333333334 0.35833333333333334
0 0.05416666666666667 0.48333333333333334 0.10416666666666667 0.23333333333333334
0 0.9875 0.25416666666666665 0.025 0.175
0 0.029166666666666667 0.1375 0.058333333333333334 0.275
0 0.1375 0.10416666666666667 0.21666666666666667 0.20833333333333334
0 0.15416666666666667 0.175 0.20833333333333334 0.275



### Save results

In [ ]:
# Store model weights and metrics
save_on_cloud(source='/content/runs/', destination='/content/drive/MyDrive/save/')

In [ ]:
# Store experiments
save_on_cloud(source='/content/model10(5_60_L)/', destination='/content/drive/MyDrive/save.SM/')